<a href="https://colab.research.google.com/github/ppiont/tensor-flow-state/blob/master/data_impute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [3]:
%cd '/gdrive/My Drive/tensor-flow-state/tensor-flow-state'

/gdrive/My Drive/tensor-flow-state/tensor-flow-state


In [0]:
import pandas as pd
df = pd.read_csv('data/combined_df.csv', index_col = 0, parse_dates = True)


In [0]:
df.rename({'speed_RWS01_MONIBAS_0021hrl0403ra': 'speed_-2', 'speed_RWS01_MONIBAS_0021hrl0409ra': 'speed_-1',\
           'speed_RWS01_MONIBAS_0021hrl0420ra': 'speed_+1', 'speed_RWS01_MONIBAS_0021hrl0426ra': 'speed_+2',\
           'flow_RWS01_MONIBAS_0021hrl0403ra': 'flow_-2', 'flow_RWS01_MONIBAS_0021hrl0409ra': 'flow_-1',\
           'flow_RWS01_MONIBAS_0021hrl0420ra': 'flow_+1', 'flow_RWS01_MONIBAS_0021hrl0426ra': 'flow_+2'\
           }, axis = 1, inplace = True)
df = df[['speed', 'flow', 'speed_-2', 'speed_-1','speed_+1', 'speed_+2', 'flow_-2', 'flow_-1', 'flow_+1', 'flow_+2', 'speed_limit']]

In [0]:
def nans(df, title = ''):
    records = len(df)
    null_records = len(df.loc[df.isnull().any(axis=1)])
    null_ratio = len(df.loc[df.isnull().any(axis=1)]) / len(df)

    print(f"\
    {title}\n\
    ---------------------------------\n\
    Number of records:      {records:,}\n\
    Number of null records:   {null_records:,}\n\
    ---------------------------------\n\
    Null ratio:                 {null_ratio:.3f}\
    ")

In [23]:
nans(df, title = 'Status before imputation')

    Status before imputation
    ---------------------------------
    Number of records:      4,727,520
    Number of null records:   448,433
    ---------------------------------
    Null ratio:                 0.095    


In [0]:
df_speed = df[['speed', 'speed_-2', 'speed_-1','speed_+1', 'speed_+2']]
df_flow = df [['flow_-2', 'flow_-1', 'flow_+1', 'flow_+2',

In [0]:
def fill_na_row_mean(df):
    row_avgs = df.mean(axis=1).values.reshape(-1,1)
    df = df.fillna(0) + df.isna().values * row_avgs
    return df

In [9]:
df3 = df2.T.fillna(df2.mean(axis=1)).T

KeyboardInterrupt: ignored

In [1]:
df3

NameError: ignored

In [0]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (30,15))
df2[:100000].plot(ax = ax)

In [0]:
pd.options.display.max_rows = 1000
df3[:1000]

In [5]:
l = df.columns.tolist()
print(l)

['speed', 'flow', 'speed_-2', 'speed_-1', 'speed_+1', 'speed_+2', 'flow_-2', 'flow_-1', 'flow_+1', 'flow_+2', 'speed_limit']


In [15]:
import numpy as np
test = df.resample('W').agg({'speed': np.mean, 'speed_-2': np.mean, 'speed_-1': np.mean, 'speed_+1': np.mean, 'speed_+2': np.mean,
                             'flow': np.sum, 'flow_-2': np.sum, 'flow_-1': np.sum, 'flow_+1': np.sum, 'flow_+2': np.sum})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: using a dict with renaming is deprecated and will be removed
in a future version.

For column-specific groupby renaming, use named aggregation

    >>> df.groupby(...).agg(name=('column', aggfunc))

  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
test2 = df2.resample('W').agg({'speed': np.mean, 'speed_-2': np.mean, 'speed_-1': np.mean, 'speed_+1': np.mean, 'speed_+2': np.mean})

In [0]:
test2

In [0]:
pd.options.display.max_rows = 1000
pd.options.display.float_format = '{:.2f}'.format
test

In [0]:
df.head(500)

In [0]:
df.loc[df.isnull().any(axis=1)]

    Number of records:      4,727,520
    Number of null records:   288,548
    ---------------------------------
    Null ratio:                 0.061    


In [0]:
import numpy as np
temp = pd.DataFrame()
df = df.assign(reference_speed = df.speed.interpolate(method = 'time'))
df = df.assign(missing = np.nan)
df.missing[df.isnull().any(axis=1)] = df.reference_speed

,speed,flow,speed_RWS01_MONIBAS_0021hrl0403ra,flow_RWS01_MONIBAS_0021hrl0403ra,speed_RWS01_MONIBAS_0021hrl0409ra,flow_RWS01_MONIBAS_0021hrl0409ra,speed_RWS01_MONIBAS_0021hrl0420ra,flow_RWS01_MONIBAS_0021hrl0420ra,speed_RWS01_MONIBAS_0021hrl0426ra,flow_RWS01_MONIBAS_0021hrl0426ra,speed_limit,reference_speed,missing
2011-01-03 00:00:00,120.000000,8.0,100.000000,8.0,120.000000,6.0,118.5,4.0,120.000000,6.0,120.0,120.000000,120.000000
2011-01-03 00:01:00,120.000000,5.0,115.000000,18.0,120.000000,6.0,118.0,5.0,104.000000,2.0,120.0,120.000000,120.000000
2011-01-03 00:02:00,111.000000,2.0,112.666667,14.0,104.000000,2.0,91.0,4.0,112.666667,8.0,120.0,111.000000,111.000000
2011-01-03 00:03:00,105.333333,10.0,118.000000,14.0,108.666667,6.0,120.0,9.0,120.000000,10.0,120.0,105.333333,105.333333
2011-01-03 00:04:00,120.000000,8.0,105.500000,21.0,120.000000,13.0,120.0,7.0,117.500000,5.0,120.0,120.000000,120.000000


In [0]:
# Fill in the null vals for reference
df = df.assign(reference_speed = dfspeed.interpolate(method = 'time'))
# Add a col with interpolated values in the missing values position (the rest null) for plotting
df = dfassign(missing = np.nan)
dfmissing[df.isnull().any(axis=1) = august.reference_speed

# Plot it (red are missing)
august[['speed', 'missing']].plot(style = ['b.', 'rx'], figsize = (16, 9), alpha = 0.6, title = 'August')
sns.despine()
plt.tight_layout()
plt.savefig(plotdir + 'August.png')


# Luckily they are not all clumped, so the interpolation should be smooth

# Now drop unnecessary speed col
august.drop('speed', axis = 1, inplace = True)

# There are no null vals left since we already interpolated reference_speed.
august['reference_speed'].isnull().any()

# Randomly null out 10% of the values in a new col target_speed that we want to test interpolation methods on
august['target_speed'] = august['reference_speed'].mask(np.random.choice([True, False], size=august['reference_speed'].shape, p = [.1,.9]))

# We now have thousands of null vals
august['target_speed'].isna().sum()

# Make a new column with only the reference values that aren't in the target col, like earlier but with the larger
# set of randomly removed values
august = august.assign(missing = np.nan)
august.missing[august.target_speed.isna()] = august.reference_speed
august.info()

# Plot it (red are missing)
august[['target_speed', 'missing']].plot(style=['b.', 'rx'], figsize=(16, 9), alpha = 0.6, title = 'August /w 10% data removed')

In [0]:
pd.set_option('display.max_rows', 500)
df.speed[df.speed.notna()].astype(int).value_counts().head(25)

In [0]:
import matplotlib.pyplot as plt
plt.figure(figsize = (16,12))
df.speed[(df.speed.notna()) & (df.speed > 70)].hist(bins = 50, grid = False)

In [0]:
df2 = df.resample('M').agg({'speed': np.mean, 'flow': np.sum})

In [0]:
plt.figure(figsize=(20,10))
plt.scatter(df.index, df.speed)

In [0]:
df['null'] = np.where(df.speed.isna(), 1, np.nan)
df['timestamp'] = df.index
df.groupby(pd.Grouper(key='timestamp', freq='M'))['null'].sum()

timestamp
2011-01-31      69.00
2011-02-28     150.00
2011-03-31     812.00
2011-04-30     336.00
2011-05-31     394.00
2011-06-30     195.00
2011-07-31    1762.00
2011-08-31    6077.00
2011-09-30     212.00
2011-10-31    1937.00
2011-11-30     165.00
2011-12-31     249.00
2012-01-31    1568.00
2012-02-29     680.00
2012-03-31    2018.00
2012-04-30    2365.00
2012-05-31     121.00
2012-06-30     722.00
2012-07-31   14973.00
2012-08-31     819.00
2012-09-30     146.00
2012-10-31    9376.00
2012-11-30      15.00
2012-12-31      44.00
2013-01-31    2973.00
2013-02-28    1267.00
2013-03-31    1873.00
2013-04-30    1036.00
2013-05-31    2219.00
2013-06-30    2184.00
2013-07-31    1265.00
2013-08-31    1378.00
2013-09-30    2254.00
2013-10-31    3050.00
2013-11-30    4116.00
2013-12-31    3663.00
2014-01-31    4774.00
2014-02-28    3897.00
2014-03-31    2245.00
2014-04-30    1537.00
2014-05-31   44282.00
2014-06-30   39657.00
2014-07-31   44357.00
2014-08-31   43324.00
2014-09-30     394.00
